## 0. Setup — RAPIDS environment

In this notebook we will use NVIDIA RAPIDS to compare
CPU vs GPU execution on the same analytical workload.

We rely on the official RAPIDS Colab utilities to install
the correct CUDA-compatible libraries.

⚠️ If no GPU is available, this notebook can still be read
but GPU cells should not be executed.


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: la ruta de destino 'rapidsai-csp-utils' ya existe y no es un directorio vacío.
/home/dalpenyes/Música/HUGBD/state-of-the-art-tools-for-big-data/exercises/session_2/nvidia/rapidsai-csp-utils/colab/pip-install.py:7: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml
Traceback (most recent call last):
  File "/home/dalpenyes/Música/HUGBD/state-of-the-art-tools-for-big-data/exercises/session_2/nvidia/rapidsai-csp-utils/colab/pip-install.py", line 18, in <module>
    pynvml.nvmlInit()
  File "/home/dalpenyes/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pynvml.py", line 2882, in nvmlInit
    nvmlInitWithFlags(0)
  File "/home/dalpenyes/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pynvml.py", line 2872, in nvmlInitWithFlags
    _nvmlCheckReturn(ret)
  File "/home/dalpenyes/.pyenv/versions/3.10.13/l

In [2]:
%pip install pandas numpy cudf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 13.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 7.9 MB/s eta 0:00:0000:0100:01
  DEPRECATION: cudf is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for cudf ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for cudf did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running install
      /home/dalpenyes/.pyenv/versions/3.10.13

In [1]:
import pandas as pd
import numpy as np
import cudf
import time

ModuleNotFoundError: No module named 'pandas'

## 1. Dataset generator

This cell controls the size and shape of the dataset.

Change the parameters and re-run the notebook to observe
how CPU and GPU performance changes.


In [ ]:
# ---- Playground parameters ----
N_ROWS = 2_000_000        # try: 500_000 | 2_000_000 | 10_000_000
N_CATEGORIES = 5          # try: 5 | 20 | 100
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

categories = [f"C{i}" for i in range(N_CATEGORIES)]

df_cpu = pd.DataFrame({
    "id": np.arange(N_ROWS),
    "category": np.random.choice(categories, size=N_ROWS),
    "value": np.random.rand(N_ROWS) * 1000,
})

df_cpu.head()


## 2. Experiment parameters

These parameters must be the same for CPU and GPU.
Change them to explore performance trade-offs.


In [ ]:
VALUE_THRESHOLD = 500     # try: 100 | 500 | 900
AGG_FUNCTION = "sum"      # try: "sum" | "mean" | "count"

## 3. CPU baseline

This is the CPU reference implementation.
You will compare GPU execution against this result.


In [ ]:
# TODO:
# - apply the filter using VALUE_THRESHOLD
# - apply the aggregation defined by AGG_FUNCTION
# - measure execution time


In [ ]:
start = time.perf_counter()

if AGG_FUNCTION == "sum":
    cpu_result = (
        df_cpu[df_cpu["value"] > VALUE_THRESHOLD]
          .groupby("category")["value"]
          .sum()
    )
elif AGG_FUNCTION == "mean":
    cpu_result = (
        df_cpu[df_cpu["value"] > VALUE_THRESHOLD]
          .groupby("category")["value"]
          .mean()
    )
elif AGG_FUNCTION == "count":
    cpu_result = (
        df_cpu[df_cpu["value"] > VALUE_THRESHOLD]
          .groupby("category")["value"]
          .count()
    )
else:
    raise ValueError("Unsupported AGG_FUNCTION")

cpu_time = time.perf_counter() - start

cpu_time, cpu_result.head()


## 4. GPU workload

Reproduce EXACTLY the same logic on GPU.

Only the execution backend should change.


In [ ]:
df_gpu = cudf.from_pandas(df_cpu)

start = time.perf_counter()

if AGG_FUNCTION == "sum":
    gpu_result = (
        df_gpu[df_gpu["value"] > VALUE_THRESHOLD]
          .groupby("category")["value"]
          .sum()
    )
elif AGG_FUNCTION == "mean":
    gpu_result = (
        df_gpu[df_gpu["value"] > VALUE_THRESHOLD]
          .groupby("category")["value"]
          .mean()
    )
elif AGG_FUNCTION == "count":
    gpu_result = (
        df_gpu[df_gpu["value"] > VALUE_THRESHOLD]
          .groupby("category")["value"]
          .count()
    )
else:
    raise ValueError("Unsupported AGG_FUNCTION")

gpu_time = time.perf_counter() - start

gpu_time, gpu_result.head()


## 5. CPU vs GPU comparison

Compare execution times and validate results.


In [ ]:
pd.DataFrame({
    "backend": ["CPU", "GPU"],
    "time_seconds": [cpu_time, gpu_time]
})


In [ ]:
cpu_result.sort_index(), gpu_result.to_pandas().sort_index()


## 6. Experiments to try

Try changing ONE parameter at a time:

- Increase N_ROWS
- Increase N_CATEGORIES
- Change VALUE_THRESHOLD
- Change AGG_FUNCTION
- Run CPU first vs GPU first

Observe:
- when GPU wins
- when GPU loses
- when results are similar
